# An Introduction to Ethical Supervised Learning

***

# Overview
In this tutorial, you will learn:

1. How to identify a discriminatory machine learning predictor
2. An introduction to fairness models in supervised learning
3. How to fix a basic discriminatory predictor with a fairness model through:
    * Understanding perfmorance metrics
    * Minimizing loss
    * Utilizing sensitivity / recall

***

# Part 1 : How to Identify a Discriminatory Predictor

## I. Introduction to TransRisk Scores

For this tutorial, we will be working with **TransRisk Scores** for four demographics. TransRisk scores aim to estimate how worthy a consumer is for credit. The scores range from 0 to 100, and are created using a variety of data points (such as history of paying back loans, how many credit card accounts you've had, employment rate, etc.) In theory, the closer a person's TransRisk score is to 100, the more likely they are to pay back a loan (similar to a FICO credit score). Banks and loan agencies typically use this kind of score combined with a **threshold value** to decide if someone should be granted a loan.<br/><br/>
For example, a bank that grants loans might set their threshold TransRisk score at 60. Then everyone with a TransRisk score of 60 or above recieves a loan, and everyone with a score below 60 automatically gets denied.<br/><br/>
The first data set you will work with shows the probability of an individual having a TransRisk score for different demographic groups. This can also be written as : 
$$ P\text{(Score} = x \text{ | race)} $$

In [2]:
import pandas as pd
%matplotlib inline
import numpy as np

In [3]:
ProbScoreEqualsXGivenRace = pd.read_csv("ProbScoreEqualsXGivenRace.csv").set_index("Score")

## II. Using Visualizations as a Guide

*** For a threshold score x, someone is given a loan if they have a TransRisk score >= x. Using this data set, create and plot a dataframe that shows the probability of an individual receiving a loan: ***
$$ P\text{(score} \geq x \text{ | race)} $$

## III. Interpretation

Using our previous example of a threshold score x = 60, take a look at your visualization. With a score of 60, what is the probability for an individual of each race to receive a loan?

In [4]:
# Show Probabilities Here

Keep in mind this isn't an indication if discrimination is occurring or not. If a bank chooses a threshold score of 60, the white demographic would be more than 4x as likely to get a loan than the black demographic. This could happen for a variety of reasons, including (but not limited to):
- The data that was used to create a TransRisk score (such as amount of bank accounts opened) might unintentionally discriminate against certain demographic groups.
- Machine learning predictors are inherently less accurate for minority populations, since smaller population sizes lead to lower predictor performance.
- For a person have a score of 60 or above in the hispanic or black demographic, they likely will need to be in a much higher percentile of their overall population as compared to the white or asian demographic

** To determine if a predictor is discriminatory, it must fail the requirements provided by a fairness model. **

***

# Part 2 : Introduction to Fairness Models

## I. What is a fairness model?

Fairness models seek to fix predictors for discrimination. Specifically, they allow data scientists to algorithmically and statistically measure how *fair* a predictor is by providing requirements to ensure *fairness*. Using these requirements, we can also determine if a predictor is *unfair* (or discriminatory).

## II. Equal Opportunity Model

This model will work well for the TransRisk dataset, because it provides simple requirements for a binary predictor. This model takes into account three factors:
1. Groups that are defined by a protected attribute (this will be the four demographic groups in our example) | Denoted as 'A'
2. The advantaged outcome group (we will discuss this in Part 3) | Denoted as 'Y = 1'
3. The binary predictor (getting granted a loan) | Denoted as '$\hat Y$ = 1' for a loan approval and '$\hat Y$ = 0' for loan denial

A Predictor is said to satisfy fairness under Equal Opportunity iff the probability of getting a positive prediction is the same for all advantaged outcome members in a protected group. <br/><br/>
$$ P(\hat Y = 1 | A = a, Y = 1) \text{ is equal for ALL protected groups a.} $$


***

# Part 3 : Fixing a Discriminatory Predictor Through the Equal Opportunity Model

## I. Understanding Performance Metrics

![Probabilities](ethicalmldiagram.png)

Here we will introduce two more data sets on these TransRisk scores. 
1. **ProbNonDefaultGivenRace**: P(NonDefault | race)
    - Represents the probability that an individual of a race is non-defaulting</li>
    - *Note that the non-defaulters represent the advantaged outcome group defined in Part 2, for all non-defaulting individuals of a race,'Y = 1'* <br/><br/>
2. **ProbNonDefaultGivenScoreGreaterXAndRace** : P(NonDefault | score >= x, race)
    - Represents the probability that an individual who has been granted a loan is actually a non-defaulter</li>
    - *Remember that with a threshold score x, everyone with a score >= x is granted a loan.*

Read them in and make sure you understand what the data represents before moving onto the next section.

In [5]:
ProbNonDefaultGivenRace = pd.read_csv("ProbNonDefaultGivenRace.csv")
ProbLoanReceiverIsNonDefault = (
    pd.read_csv("ProbNonDefaultGivenScoreGreaterXAndRace.csv")
    .set_index('TransRisk Score'))

***

### <center> The General Multiplication Rule: </center>
<center> 
**Definition : ** For any two events, we can say that
<br/><br/>
$$ P(A \text{ and } B) = P(A) * P(B | A)$$
<br/>
*"The probability that both A and B occur = the probability that A occurs multiplied by the probability that B occurs, given that A has occured."*
</center>

***

To work with the conditions introduced by the **Equal Opportunity Model**, we need to find

$$ P(\hat Y = 1 | A = a, Y = 1)$$

For our case study, this would be the Probability that someone has received a loan, given their race and the fact that they are non-defaulting. Also stated:

$$ P(\text{score} \geq x | \text{ race, NonDefault)}$$ 

*If you look at the visual table of probabilities above, this is shown with the **PURPLE** boxes.*

Using your knowledge of the multiplication rule with all three data sets you have been given on TransRisk scores, create a DataFrame that shows this probability for each TransRisk score and demographic, then plot it to see what this data looks like.

** Now, restructure this dataframe so the sensitivity values are the index, and each demographic has one TransRisk Score (threshold value) associated with each sensitivity **

## II. Minimizing Loss

For supervised learning predictors, one way to detect errors is to calculate the difference between the actual output and the predictor's output. The function used to compute the error of a predictor is called a **Loss Function**. For our dataset, we calculate loss in a similar manner to how banks giving loans would. Loss is calculated using False Positives (FP) and False Negatives (FN), which means every loss value can be associated with a precision value. Generally, banks strive for a maximum loss of 0.18, which correlates to a precision of 0.82. So, we want to make the ** *Precision = 0.82* **.

$$ \text{Precision   ==   P(Y} = 1 | \hat Y = 1) \text{  ==   P(NonDefault | score} \geq x) $$

Here we provide another dataset that shows the percentage of each demographic in the total sample size, or rather P(race). Using this information and the multiplication rule, find the precision for every demographic at every sensitivity value. So you should create four new columns in your sensitivity dataframe from before, one for each demographic precision.

In [7]:
# This represents P(race)
ProbOfBeingRace = pd.read_csv("ProbOfBeingRace.csv").set_index("Demographic")

## III. Utilizing Sensitivity

We finally have all of the peices to fix the discriminatory predictor using the Equal Opportunity Model. As you just learned in Section II, our desired loss directly leads to a precision of 0.82. Looking back at the requirements for Equal Opportunity, we can see it requires that $ P(\hat Y = 1 | A = a, Y = 1)$ is equal for ALL protected groups a. This can also be stated that the **true positive rate** is equal for ALL protected groups a. Or even further, that the ** *Sensitivity* ** is equal for ALL protected groups a.

If you look back at your plot of sensitivity values, you can see that choosing one sensitivity value (drawing a horizontal line on your plot) would yeild four threshold values. But where do we draw this line? Which singular sensitivity value do we want?

As we saw in the last section, we choose the sensitivity value that gives us a total precision of 0.82. Now you simply need to find the threshold scores associated with that sensitivity value for each demographic. These scores represent the threshold that needs to be set for that particular demographic in order to satisfy 'fairness'. Finish by recording the final four threshold values needed to satisfy the Equal Opportunity Model.

** First: Aggregate the four precisions at each sensitivity value for a TOTAL PRECISION column in your dataframe. **

*** What four sensitivity values are associated with a precision of 0.82? *** (put answer below)

***

# Results & Conclusions

** *Congratulations!* **

You have successfully taken a discriminatory predictor and utilized a machine learning fairness model to make it into a 'fair' predictor. 
- Through visualizing the probability of a beneficial outcome, you could see that the probabilities for different demographics to receive loans weren't statistically close. This wasn't necessarily a measure of discrimination.
- The Equal Opportunity Model provided a clear requirement to ensure fairness. As you learned later, this meant that the sensitivity (the true positive rate) of the predictor had to be the same for all races. Since one threshold value resulted in four different sensitivities, you were able to determine that discrimination will occur unless you choose one threshold per demographic.
- Using loss minimization, you were able to use precision to find the single sensitivity that provided the least amount of loss.
- Finally, you discovered four new threshold values to be used; one for each demographic. Ensuring fairness and fixing a discriminatory predictor.
<br/><br/>

<center>
*What other commonly found predictors do you think might be inherently discriminatory?*
</center>


***

TransRisk data and non-discriminatory analysis courtesy of https://arxiv.org/pdf/1610.02413.pdf